In [2]:
import pandas as pd 
import numpy as np

In [3]:
df_raw = pd.read_csv('C:\\Users\\USER\\Desktop\\FYP\\fyp-sandbox-2\\Data\\RAW_DATA\\DRUGS/DRUG24Q4.txt', sep='$')
df_raw

C:\Users\USER\AppData\Local\Temp\ipykernel_3100\1720144281.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv('C:\\Users\\USER\\Desktop\\FYP\\fyp-sandbox-2\\Data\\RAW_DATA\\DRUGS/DRUG24Q4.txt', sep='$')


,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,lot_num,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq
0,100100247,10010024,1,PS,ADALIMUMAB,ADALIMUMAB,1,Subcutaneous,NaN,NaN,NaN,Y,NaN,Not Available,NaN,125057.0,40.0,MG,NaN,QOW
1,100100247,10010024,2,SS,ADALIMUMAB,ADALIMUMAB,1,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,125057.0,NaN,NaN,NaN,NaN
2,100100247,10010024,3,SS,CYCLOSPORINE,CYCLOSPORINE,1,Unknown,NaN,NaN,NaN,D,NaN,Not Available,NaN,65003.0,NaN,NaN,NaN,NaN
3,100100247,10010024,4,SS,REMICADE,INFLIXIMAB,1,Intravenous (not otherwise specified),6 infusions at 8 weeks interval,NaN,NaN,Y,NaN,Not Available,NaN,NaN,NaN,NaN,NaN,NaN
4,100100247,10010024,5,SS,ETANERCEPT,ETANERCEPT,1,Subcutaneous,NaN,NaN,NaN,Y,NaN,Not Available,NaN,NaN,100.0,MG,NaN,/WK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030933,99696784,9969678,12,SS,DAPTOMYCIN,DAPTOMYCIN,1,Unknown,"500 mg, Daily",NaN,NaN,Y,NaN,NaN,NaN,NaN,500.0,MG,NaN,NaN
2030934,99696784,9969678,13,SS,PANTOPRAZOLE SODIUM I.V.,PANTOPRAZOLE SODIUM,1,Unknown,"40 mg, UNK",NaN,NaN,U,U,unk,NaN,NaN,40.0,MG,Injection,NaN
2030935,99696784,9969678,14,C,ACETAMINOPHEN\CODEINE PHOSPHATE,ACETAMINOPHEN\CODEINE PHOSPHATE,1,Unknown,"500/30 mg, daily",NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2030936,99696784,9969678,15,C,METHYLPREDNISOLONE,METHYLPREDNISOLONE,1,Unknown,"80 mg, bid",NaN,NaN,Y,NaN,NaN,NaN,NaN,80.0,MG,NaN,Q12H


### Creating a logic for replacing the missing prod_ai values

In [4]:
df_raw.isna().sum()

primaryid              0
caseid                 0
drug_seq               0
role_cod               0
drugname               7
prod_ai            30030
val_vbm                0
route             641361
dose_vbm          619190
cum_dose_chr     1993811
cum_dose_unit    1993811
dechal            822598
rechal           1853194
lot_num          1530761
exp_dt           2029280
nda_num          1378374
dose_amt         1192311
dose_unit        1192311
dose_form         994497
dose_freq        1606951
dtype: int64

In [5]:
df_raw.columns

Index(['primaryid', 'caseid', 'drug_seq', 'role_cod', 'drugname', 'prod_ai',
       'val_vbm', 'route', 'dose_vbm', 'cum_dose_chr', 'cum_dose_unit',
       'dechal', 'rechal', 'lot_num', 'exp_dt', 'nda_num', 'dose_amt',
       'dose_unit', 'dose_form', 'dose_freq'],
      dtype='object')

In [6]:
#First let's remove the unwanted cols
df_raw_2 = df_raw[['primaryid', 'caseid', 'drug_seq', 'role_cod', 'drugname', 'prod_ai']]
df_raw_2.isna().sum()

primaryid        0
caseid           0
drug_seq         0
role_cod         0
drugname         7
prod_ai      30030
dtype: int64

In [7]:
df_raw_2[df_raw_2['drugname'].isna() == True]

,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai
945010,244269481,24426948,2,C,NaN,UNSPECIFIED INGREDIENT
1991280,247842001,24784200,2,C,NaN,UNSPECIFIED INGREDIENT
1991281,247842001,24784200,3,C,NaN,UNSPECIFIED INGREDIENT
1991282,247842001,24784200,4,C,NaN,UNSPECIFIED INGREDIENT
1991284,247842011,24784201,2,C,NaN,UNSPECIFIED INGREDIENT
1991285,247842011,24784201,3,C,NaN,UNSPECIFIED INGREDIENT
1991286,247842011,24784201,4,C,NaN,UNSPECIFIED INGREDIENT


If the drugname is missing, it can be taken out (dropped). Since the drug is unspecified, it would not affect the analysis or anything by a one particular case having missed drugs

In [8]:
#Step 01: remove any records without the drugname
cods = df_raw_2[df_raw_2['drugname'].isna() == True]['primaryid']
temp_1= df_raw[df_raw['primaryid'].isin(cods)]
temp_1[temp_1['drugname'].isna()==True]

,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,lot_num,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq
945010,244269481,24426948,2,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991280,247842001,24784200,2,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991281,247842001,24784200,3,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991282,247842001,24784200,4,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991284,247842011,24784201,2,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991285,247842011,24784201,3,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991286,247842011,24784201,4,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


So it's confirmed that the records with no drugname have no other info. And it's safe to remove all of them